# Setup notebook

In [1]:
import geopandas as gp
import bw2regional as br
import bw2data as bd
import bw2calc as bc
from pathlib import Path
import numpy as np

# Setup and reset project

In [2]:
bd.projects.set_current("case-study")

In [3]:
for db_name in list(bd.databases):
    del bd.databases[db_name]
    
for meth in list(bd.methods):
    del bd.methods[meth]
    
for gc in list(br.geocollections):
    del br.geocollections[gc]    

# Setup database

In [4]:
db = bd.Database("case")

#declare the geolocation of the map of 1 single spatial delineation for LCI and lCIA
db.register(geocollections=["regions"])

# Setup geocollections

In [5]:
DATA_PATH = (Path.cwd().parent / "files").resolve()
DATA_PATH, list(DATA_PATH.iterdir())

(PosixPath('/Users/cmutel/Code/brightway/from-the-ground-up/files'),
 [PosixPath('/Users/cmutel/Code/brightway/from-the-ground-up/files/CF_polygones.gpkg'),
  PosixPath('/Users/cmutel/Code/brightway/from-the-ground-up/files/rice_provinces.gpkg')])

In [6]:
br.geocollections["watersheds"] = {
    'filepath': str(DATA_PATH / 'CF_polygones.gpkg'),
    # 'layer': 'CF_margina', 
    'field': 'ID',
}
br.geocollections["watersheds"] 

{'filepath': '/Users/cmutel/Code/brightway/from-the-ground-up/files/CF_polygones.gpkg',
 'field': 'ID',
 'sha256': 'e7f3c957ca998c4c90a03afe5b657cb55669f5515bc6e0f81aa917aa0ed565a2',
 'kind': 'vector'}

In [7]:
br.geocollections["regions"] = {
    'filepath': str(DATA_PATH / 'rice_provinces.gpkg'),
    'field': 'iso_3166_2',
}
br.geocollections["regions"] 

{'filepath': '/Users/cmutel/Code/brightway/from-the-ground-up/files/rice_provinces.gpkg',
 'field': 'iso_3166_2',
 'sha256': '1e2aab70520e4cd748a8324f278511234ae57a5375a7fdf33c002c5d1f1a0f2f',
 'kind': 'vector'}

# Setup regionalized LCIA method

In [8]:
data = {
    'code': 'freshwater',
    'name': 'freshwater consumption',
    'location': 'GLO',
    'unit': 'm3',
    'type': 'stressor'
}

freshwater = db.new_activity(**data)
freshwater.save()

## Import regionalized map

Normally we would use the function `br.import_regionalized_cfs`, but the provided maps has regions where the CF is `Null`. This will break things, so we need to do this import ourselves. Luckily, it isn't too difficult.

In [9]:
def import_regionalized_cfs_with_geopandas(geocollection, method_tuple, mapping, scaling_factor=1, global_cfs=None):
    assert (geocollection in br.geocollections 
            and br.geocollections[geocollection].get('kind') == 'vector'
            and "field" in br.geocollections[geocollection]
           )
    gdf = gp.read_file(br.geocollections[geocollection]['filepath'])
    id_label = br.geocollections[geocollection]["field"]
    
    method = bd.Method(method_tuple)
    method.metadata['geocollections'] = [geocollection]
    bd.methods.flush()
    
    data = []
    if global_cfs:
        data.extend(global_cfs)

    for index, feature in gdf.iterrows():
        for field_label, biosphere_flows in mapping.items():
            if feature[field_label] is None or np.isnan(feature[field_label]):
                continue
            else:
                for flow in biosphere_flows:
                    data.append((
                        flow, 
                        float(feature[field_label]) * scaling_factor, 
                        (geocollection, feature[id_label])
                    ))
    
    method.write(data)

In [10]:
bd.Method(("water consumption", "watershed scale")).register(unit="PDF")
import_regionalized_cfs_with_geopandas(
    "watersheds", 
    ("water consumption", "watershed scale"), 
    {'EF_margina': [freshwater.key]}
)

In [11]:
bd.Method(("water consumption", "watershed scale")).load()[:20]

[(('case', 'freshwater'), 6.243519e-09, ('watersheds', 54)),
 (('case', 'freshwater'), 7.974224e-09, ('watersheds', 35)),
 (('case', 'freshwater'), 3.777942e-09, ('watersheds', 49)),
 (('case', 'freshwater'), 7.909293e-09, ('watersheds', 73)),
 (('case', 'freshwater'), 4.475176e-09, ('watersheds', 112)),
 (('case', 'freshwater'), 1.6187929e-08, ('watersheds', 155)),
 (('case', 'freshwater'), 1.2216529e-08, ('watersheds', 2832)),
 (('case', 'freshwater'), 1.2816316e-08, ('watersheds', 2963)),
 (('case', 'freshwater'), 2.567833e-09, ('watersheds', 5638)),
 (('case', 'freshwater'), 2.291942e-09, ('watersheds', 5708)),
 (('case', 'freshwater'), 3.430738e-09, ('watersheds', 15425)),
 (('case', 'freshwater'), 1.55811e-10, ('watersheds', 23897)),
 (('case', 'freshwater'), 2.7739e-10, ('watersheds', 23844)),
 (('case', 'freshwater'), 6.08337e-10, ('watersheds', 23808)),
 (('case', 'freshwater'), 1.886911e-09, ('watersheds', 352)),
 (('case', 'freshwater'), 4.51159e-10, ('watersheds', 301)),
 (

# Calculate intersection

We can just use geopandas instead of the pandarus library.

In [12]:
def write_intersection(first, second, overwrite=False):
    if (first, second) in br.intersections and not overwrite:
        raise ValueError("Intersection already exists")
    
    for gc in (first, second):
        assert (gc in br.geocollections 
            and br.geocollections[gc].get('kind') == 'vector'
            and "field" in br.geocollections[gc]
           )
    assert br.geocollections[first]['filepath'] != br.geocollections[second]['filepath']
    
    df1 = gp.read_file(br.geocollections[first]['filepath'])
    df2 = gp.read_file(br.geocollections[second]['filepath'])
    id1 = br.geocollections[first]["field"]
    id2 = br.geocollections[second]["field"]

    assert id1 != id2, "Conflicting ID labels"
    
    intersection = gp.overlay(df1, df2)
    areas = intersection.to_crs("esri:54009").area  # World Mollweidge in square meters
    
    data = []
    for index, feature in intersection.iterrows():
        data.append((feature[id1], feature[id2], areas[index]))
    
    obj = br.Intersection((first, second))
    obj.register()
    obj.write(data)
    obj.create_reversed_intersection()

In [13]:
write_intersection("regions", "watersheds")

In [14]:
br.Intersection(("regions", "watersheds")).load()

[('CN-62', 10944, 587710246.891498),
 ('CN-62', 10784, 2813916857.421469),
 ('CN-62', 10976, 5624540.639780564),
 ('CN-62', 10781, 4505834025.288664),
 ('CN-62', 10945, 7706500913.38276),
 ('CN-62', 10979, 6601614202.278695),
 ('CN-62', 10949, 913736681.8139017),
 ('CN-62', 11263, 5083478143.000593),
 ('CN-62', 11347, 149965830.59423748),
 ('CN-62', 11344, 7290229563.897603),
 ('CN-62', 11349, 71420084.42502172),
 ('CN-62', 12236, 573727299.6146897),
 ('CN-62', 12359, 2245344177.179704),
 ('CN-62', 11641, 273397943.06686234),
 ('CN-62', 11627, 420060145.33159363),
 ('CN-62', 11762, 178239558.02328178),
 ('CN-62', 11704, 14334723696.459213),
 ('CN-62', 11733, 51190037.835351355),
 ('CN-62', 11735, 666630809.7632365),
 ('CN-62', 11772, 177440040.06315106),
 ('CN-62', 12516, 231404400.70900425),
 ('CN-62', 12518, 201165518.96016642),
 ('CN-62', 11844, 542497258.1392629),
 ('CN-62', 12235, 79301029817.98105),
 ('CN-62', 11919, 2732084619.9408875),
 ('CN-62', 12925, 38241150.36060973),
 ('C

# Setup inventory graph

In [17]:
data = {
    'code': 'rice india',
    'name': 'rice production india',
    'location': ('watersheds','IN'),
    'unit': 't'
}

rice_IN = db.new_activity(**data)
rice_IN.save()

In [18]:
data = {
    'code': 'rice china',
    'name': 'rice production china',
    'location': ('watersheds','CN'),
    'unit': 't'
}

rice_CN = db.new_activity(**data)
rice_CN.save()

In [19]:
data = {
    'code': 'rice USA',
    'name': 'rice production USA',
    'location': ('watersheds','US'),
    'unit': 't'
}

rice_US = db.new_activity(**data)
rice_US.save()

In [20]:
data = {
    'code': 'pesticide',
    'name': 'pesticide production',
    'location': 'GLO',
    'unit': 't'
}

pesticide = db.new_activity(**data)
pesticide.save()

In [23]:
rice_IN.new_exchange(#modified
    amount=100, 
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='biosphere',
    input=freshwater,
).save()

In [24]:
rice_CN.new_exchange(#modified
    amount=100, 
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='biosphere',
    input=freshwater,
).save()

In [25]:
rice_US.new_exchange(#modified
    amount=100, 
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='biosphere',
    input=freshwater,
).save()

In [26]:
pesticide.new_exchange(#modified
    amount=100, 
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='biosphere',
    input=freshwater,
).save()

In [27]:
rice_IN.new_exchange(#modified
    amount=1,
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='technosphere',
    input=pesticide,
).save()

In [28]:
rice_CN.new_exchange(#modified
    amount=1, 
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='technosphere',
    input=pesticide,
).save()

In [29]:
rice_US.new_exchange(#modified
    amount=1, 
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='technosphere',
    input=pesticide,
).save()

In [94]:
sg_lca = bc.LCA({rice_US: 1}, ("water consumption", "watershed scale")) # site generic
sg_lca.lci()
sg_lca.lcia()
sg_lca.score

2000.0

In [95]:
sg_lca.characterization_matrix.toarray()

array([[10.]])

In [96]:
lca = br.OneSpatialScaleLCA({rice_IN: 1}, ("water consumption", "watershed scale"))

In [97]:
lca.lci()
lca.lcia()
lca.score

500.0

In [28]:
pd.DataFrame([
    {
        'score': sg_lca.score, 
        'inv': sg_lca.inventory.sum(), 
        # 'heat': lca.technosphere_matrix[lca.dicts.product[ng.id], lca.dicts.activity[cf.id]],
        'char': sg_lca.characterization_matrix.sum()
    } #for _, _ in zip(lca, range(10))
])

,score,inv,char
0,2000.0,200.0,10.0
